# Update address point PIDNs from parcel layer
To do the above, this script will:
1. Copy a stripped down version of parcel data to an in-memory feature class
2. Use an intersect operation on the above to create a layer of overlapping parcel polygons, to document corrupt data which will be excluded as a source for address point updates
3. Copy stripped down address point data into in-memory feature class
4. Do a spatial join of our two in-memory feature classes, outputting to a 3rd in-memory feature class
5. Scan and filter the above feature class to create a Python dictionary which contains PIDN mismatches for parcels and their contained address points
6. Give user a total count of mismatches and allow viewing of these
7. Allow user to perform a bulk update of address points with mismatches

## Preparation

In [72]:
# following list needs to be defined w/ fields in this order,
# script won't work at all if PIDN is not first
SSAP_DesiredFieldList = ['PIDN', 'GlobalID', 'Add_Number', 'St_PreDir', 'St_Name',
     'St_PosTyp', 'St_PosDir', 'Inc_Muni' ]

import arcpy
thisProj = arcpy.mp.ArcGISProject('CURRENT')

In [73]:
print()
thisMap = thisProj.activeMap
if not thisMap:
    sys.stdout.write('\n'*4 + '    ' + '#'*73)
    print('''
    ###                                                                   ###
    ###  No active map. Check that a map with parcels and SSAPs is open,  ###
    ###  click on the map pane, and re-run this cell.                     ###
    ###                                                                   ###''')
    print('    ' + '#'*73 + '\n'*4)


In [74]:
print()
if not thisMap:
    raise UserWarning('See message in previous cell and fix.')

Get references to our source layers. For simplicity, we'll just use the layer names from our map and let our script find the full path/url to the data source.
### Change values in following cell as needed to layer names per your active map.
These layer names will be validated in the cell after the next one.

In [75]:
parcelLayerName = 'Parcels'  # CHANGE AS NEEDED
ssapLayerName = 'SSAP_YorkCityTestCopy_onAGOL'  # CHANGE AS NEEDED

### Following cell gets & validates parcel and SSAP layers

In [76]:

parcelLayer = ssapLayer = None
try:
    parcelLayer = thisMap.listLayers(parcelLayerName)[0]
except:
    print(f'Layer {parcelLayerName} not found in active map. Check spelling/punctuation')
try:
    ssapLayer = thisMap.listLayers(ssapLayerName)[0]
except:
    print(f'Layer {ssapLayerName} not found in active map. Check spelling/punctuation')

if parcelLayer and ssapLayer:
    print(f'''
Parcels data source: 
  {parcelLayer.dataSource}
SSAP data source: 
  {ssapLayer.dataSource}
    ''')

if parcelLayer:
    parcelLayerHasPIDN = False
    try:
        arcpy.ListFields(parcelLayer.dataSource, 'PIDN')[0]
        parcelLayerHasPIDN = True
    except:
        print(f'Field PIDN is missing from parcel layer {parcelLayerName}')

if ssapLayer:
    ssapFieldList = [ field.name for
                             field in arcpy.ListFields(ssapLayer.dataSource) ]
    ssapLayerHasReqdFields = True # unless changed below
    for thisField in SSAP_DesiredFieldList:
        if not (thisField in ssapFieldList):
            ssapLayerHasReqdFields = False
            print(f'Field {thisField} missing from {ssapLayerName}')
if parcelLayer and ssapLayer and parcelLayerHasPIDN and ssapLayerHasReqdFields:
    print('''
    Layers validated.  Continue.
    
    ''')
else:
    print('''
    ####################################################################
    ###  This script will not work. Correct previous cell, or stop.  ###
    ####################################################################
    ''')

Parcels data source: 
  https://arcweb1.ycpc.org/server/rest/services/OPEN_DATA/Parcels/MapServer/0
SSAP data source: 
  https://services.arcgis.com/1nZgnYZACdwzrFHH/arcgis/rest/services/SSAP_YorkCityTestCopy/FeatureServer/0


    Layers validated.  Continue.
    
    


## Use definition queries in ssap & parcel layers?
Set to True or False in the following cell.
If 'True', definition queries will be honored. This also means that if any parcels or SSAPs have been selected in these layers, *only* these parcels/SSAPs will be used to correct PIDNs in the rest of the workflow.

In [77]:
USE_DEFINITION_QUERIES = True

## Get parcel data in-memory (curr. takes 1-2 min)

Field mapping can be a pain, but let's use a function to handle the simple case of our output having a subset of input fields, no name changes. We're pulling a lot of data in memory for speed, let's only get what we need.

In [78]:
# RUNNING THIS CELL WILL TAKE 1-2 MINUTES

tempParcelLayer = 'tempParcelPIDNonly'
tempParcelFeat = 'memory/' + tempParcelLayer

from arcpy.conversion import ExportFeatures

def fieldMapsForSubset(featClass, desiredFieldNameList):
    if not ( type(featClass) is str ):
        featClass = featClass.dataSource
    simpleFieldMaps = arcpy.FieldMappings()
    for thisFieldName in desiredFieldNameList:
        thisFM = arcpy.FieldMap()
        thisFM.addInputField(featClass, thisFieldName)
        thisFM.outputField.name = thisFieldName
        simpleFieldMaps.addFieldMap(thisFM)
    return simpleFieldMaps

parcelFieldSubsetMaps = fieldMapsForSubset(parcelLayer, ['PIDN'])
print('Getting parcels ...')
# using .dataSource below is simple way to ignore selection, ensure completeness
if USE_DEFINITION_QUERIES:
    ExportFeatures(parcelLayer, tempParcelFeat,
      field_mapping=parcelFieldSubsetMaps)
else:
    ExportFeatures(parcelLayer.dataSource, tempParcelFeat,
      field_mapping=parcelFieldSubsetMaps)
    
parcelCount = int(arcpy.management.GetCount(tempParcelFeat)[0])
print('Got {} parcels.  Done'.format(parcelCount))

Getting parcels ...
Got 174296 parcels.  Done


## Find overlapping parcel polygons
We want to know, since parcels shouldn't overlap. We'll exclude these when assigning PIDNs to SSAPs.  We'll save the overlaps to an in-memory feature class; the user can export to a file/.gdb if they want.
Running this process directly on the AGOL layers would take forever. On our in-memory feature classes it takes under ten seconds.

In [79]:
overlapFeat = 'memory/OverlappingParcels'
print('Starting intersect ...')
arcpy.analysis.PairwiseIntersect(tempParcelFeat, overlapFeat)
overlapCount = int(arcpy.management.GetCount(overlapFeat)[0])
print(f'{overlapCount} overlapping parcel features found.')
if overlapCount == 0:
    print(f'Will delete empty layer {overlapFeat}')
    arcpy.management.Delete(overlapFeat)

Starting intersect ...
1058 overlapping parcel features found.


## Remove overlaps from in-memory Parcels layer
Handling overlaps can be complicated. The 'Intersect' geoprocess finds them, but the count of polygons found this way *may* include hard-to-visualize artifacts/slivers etc, which may be hard to select for deletion w/o also deleting usable parcels from our temporary layer. Below, we will only delete parcel polygons which are 'WITHIN' or 'CONTAINS' an overlap feature created above.

In [80]:
if not overlapCount:
    print('No overlaps found in previous step, skipping ...')
else:
    # using feature class, not layer, ignore selection   v
    parcelCount = int(arcpy.management.GetCount(tempParcelFeat)[0])
    arcpy.management.SelectLayerByLocation(tempParcelLayer,
        overlap_type='CONTAINS', select_features=overlapFeat)
    arcpy.management.SelectLayerByLocation(tempParcelLayer,
        selection_type='ADD_TO_SELECTION',
        overlap_type='WITHIN', select_features=overlapFeat)

    # NOTE -----------------------------------------------v
    selectCount = int(arcpy.management.GetCount(tempParcelLayer)[0])
    if (selectCount == 0) or (selectCount == parcelCount):
        print('No overlaps selected. Already removed?')
    else:
        print(f'''
    {selectedCount} overlapping features to be deleted from {tempParcelFeat}
    which has {parcelCount} features in total.''')
        arcpy.management.DeleteFeatures(tempParcelLayer)
        print('Done.')



    701 overlapping features to be deleted from memory/tempParcelPIDNonly
    which has 174296 features in total.
Done.


## Get SSAP data in-memory (1-2 minutes?)
Our only SSAP identifier that is unique and guaranteed to exist is the GlobalID. So we want to preserve that when we copy this layer into memory, and when we join these points with parcels, so that after we find mismatches, we can use it to look up the SSAPs we want to update. GlobalID may be overwritten for many geoprocessing operations unless we set the flag to preserve it, which we do before copying SSAPs into memory.

In [81]:
# RUNNING THIS CELL WILL TAKE LONGER THAN GETTING PARCEL DATA
# BUT FOR ME, STILL UNDER 2 MINUTES

from arcpy.management import CalculateField

tempSSAP_Feat = 'memory/tempSSAP'
SSAP_FieldSubsetMaps = fieldMapsForSubset(ssapLayer, SSAP_DesiredFieldList)
print('Getting SSAPs ...')

arcpy.env.preserveGlobalIds = True

if USE_DEFINITION_QUERIES:
    ExportFeatures(ssapLayer, tempSSAP_Feat,
      field_mapping=SSAP_FieldSubsetMaps)
else:
    ExportFeatures(ssapLayer.dataSource, tempSSAP_Feat,
      field_mapping=SSAP_FieldSubsetMaps)
    
# THIS NEXT STEP IS HOW WE CAN PRESERVE THIS GlobalID DURING UPCOMING JOIN
CalculateField('tempSSAP', 'SSAP_GlobalID', field_type='TEXT',
    expression='!GlobalID!.lower()')
# NOTE: On AGOL, GlobalIDs are   ^  lower case

ssapCount = int(arcpy.management.GetCount(tempSSAP_Feat)[0])
print(f'Got {ssapCount} SSAPs.  Done')

Getting SSAPs ...
Got 782 SSAPs.  Done


## Join SSAP w/ Parcels
Preserving GlobalID, this time by calculating it into another field. Note that the other field type is 'TEXT' instead of GUID, since on AGOL GlobalIDs seem to be lower case, and our SQL select will fail if we try to select w/ an uppercase value.

In [82]:
tempParcelSSAP_JoinLayer = 'tempParcelSSAP_Join'
tempParcelSSAP_JoinFeat = 'memory/' + tempParcelSSAP_JoinLayer

arcpy.env.preserveGlobalIds = True
print('Joining ...')
arcpy.analysis.SpatialJoin(tempSSAP_Feat, tempParcelFeat,
    tempParcelSSAP_JoinFeat, match_option='WITHIN',
    join_type='KEEP_COMMON')

joinCount = int(arcpy.management.GetCount(tempParcelSSAP_JoinFeat)[0])
print(f'{joinCount} join features created')
    
print('Deleting temporary SSAP and Parcel copy layers')
arcpy.management.Delete(tempParcelFeat)
arcpy.management.Delete(tempSSAP_Feat)

Joining ...
782 join features created
Deleting temporary SSAP and Parcel copy layers


<Result 'true'>

# Create Python dictionary list from mismatches in join layer

In [83]:
from arcpy.da import SearchCursor

joinFieldList = [ 'PIDN_1' ] + SSAP_DesiredFieldList
joinFieldList.remove('GlobalID')
joinFieldList += [ 'SSAP_GlobalID' ]
PIDN_CorrectionList = []

arcpy.env.preserveGlobalIds = True
with SearchCursor(tempParcelSSAP_JoinFeat, joinFieldList) as theCursor:
    for thisRow in theCursor:
        PIDNperParcel = thisRow[0] # PIDN_1
        ssapPIDN = thisRow[1] # PIDN
        if ssapPIDN == PIDNperParcel:
            continue
        dataDict = {}
        valueList = list(thisRow)  # make it 'poppable'
        for thisField in joinFieldList:
            fieldValue = valueList.pop(0)
            if thisField == 'PIDN':
                thisField = 'CurrentPIDN'
            elif thisField == 'PIDN_1':
                thisField = 'CorrectPIDN'
            dataDict[thisField] = fieldValue
        PIDN_CorrectionList += [ dataDict ]

if PIDN_CorrectionList:
    mismatchFound = True
    print(f'{len(PIDN_CorrectionList)} PIDN mismatches found.')
else:
    mismatchFound = False
    print('No PIDN mismatches found.')

print('Deleting temporary join layer')  
arcpy.management.Delete(tempParcelSSAP_JoinFeat)
print('Done')

92 PIDN mismatches found.
Deleting temporary join layer
Done


## Look at our list of corrections

In [84]:
PRINT_LINE_LIMIT = 1000

addressFields = SSAP_DesiredFieldList.copy()
addressFields.remove('PIDN')
addressFields.remove('GlobalID')

def addressStringFromCorrectionEntry(fixDict):
    addrString = ''
    for thisField in addressFields:
        if fixDict[thisField]:
            thisString = str(fixDict[thisField]).strip()
            if thisString:
                addrString += thisString + ' '
    return addrString
    
PIDN_Issue = False

if not mismatchFound:
    print('No mismatches were found -- skipping ...')
else:
    printLineCount = 0
    for thisFix in PIDN_CorrectionList:
        fixString = '{} => {}'.format(
            thisFix['CurrentPIDN'], thisFix['CorrectPIDN'])
        addrString = addressStringFromCorrectionEntry(thisFix)
        if printLineCount <= PRINT_LINE_LIMIT:
            print(f'PIDN: {fixString}  {addrString}')
        newPIDN = thisFix['CorrectPIDN']
        
        # bad PIDNs won't happen often, so we'll print regardless of line limit
        if len(newPIDN) != 13:
            PIDN_Issue = True
            print(f'  *** new PIDN >{newPIDN}< is not 13 characters')
        if ' ' in newPIDN:
            PIDN_Issue = True
            print(f'  *** new PIDN >{newPIDN}< contains a space')
        printLineCount += 1
    
    print('Addresses above rough guide: modifiers not included')
    
    print(f'{len(PIDN_CorrectionList)} PIDN mismatches')
    if printLineCount > PRINT_LINE_LIMIT:
        print(f'''
        Did not print all mismatches, over PRINT_LINE_LIMIT = {PRINT_LINE_LIMIT}
        But will apply *all* changes to SSAP layer if you continue''')
        
    if PIDN_Issue:
        print('''
Note that one of the new PIDNs had a space or was not 13 characters
        ''')
    else:
        print('''
        All PIDN corrections from parcel layer look reasonable
          (13 chars w/ no spaces)
        ''')

PIDN: 0713603001300 => 0713603001400  521 East Philadelphia Street York City 
PIDN: 1237207005300 => 1237207005200  830 East Philadelphia Street York City 
PIDN: 11312060005A0 => 11312060005B0  475 West Philadelphia Street York City 
PIDN: 0202902001500 => 0202902001400  351 East Philadelphia Street York City 
PIDN: 1237608001600 => 1237608001700  691 East Philadelphia Street York City 
PIDN: 1237507001000 => 1237507001200  723 East Philadelphia Street York City 
PIDN: 1237507001700 => 1237507001800  737 East Philadelphia Street York City 
PIDN: 1237908003200 => 1237908003300  510 East Philadelphia Street York City 
PIDN: 1237407006000 => 1237407006100  708 East Philadelphia Street York City 
PIDN: 1237708003800 => 1237708003900  658 East Philadelphia Street York City 
PIDN: 1237507001400 => 1237507001500  731 East Philadelphia Street York City 
PIDN: 1237708005000 => 1237708005100  688 East Philadelphia Street York City 
PIDN: 1237708004600 => 1237708004700  678 East Philadelphia Stre

# Run following cell for information on applying above changes

In [78]:
ENABLE_UPDATE = False
UPDATE_ONE_ONLY = False
UPDATE_MINUTES_MAXIMUM = None
print(f'''
To update PIDNs per above for {ssapLayer.name} layer
({ssapLayer.dataSource})

Run the cell after 'Enable full updates' *or* 'Enable *One* update',
*then* run the following cell
''')


To update PIDNs per above for SSAP_YorkCityTestCopy_onAGOL layer
(https://services.arcgis.com/1nZgnYZACdwzrFHH/arcgis/rest/services/SSAP_YorkCityTestCopy/FeatureServer/0)

Run the cell after 'Enable full updates' *or* 'Enable *One* update',
*then* run the following cell



In [82]:
########################################
###                                  ###
###  RUN THIS CELL TO APPLY UPDATES  ###
###                                  ###
########################################
try:
    ENABLE_UPDATE
except:
    print('Run following cell to enable updates then come back here.')
    ENABLE_UPDATE = False
    
if not ENABLE_UPDATE:
    print('Updates not enabled ... skipping cell')
else:
    UPDATE_LIMIT = None
    if UPDATE_ONE_ONLY:
        UPDATE_LIMIT = 1
    elif UPDATE_TEN_ONLY:
        UPDATE_LIMIT = 10
    try:
        PIDN_CorrectionList
        PCL_exists = True
    except:
        PCL_exists = False
        print('Error: no PIDN_CorrectionList -- cells run out of sequence?')
    if PCL_exists and (not mismatchFound):
        print('No mismatches were found -- nothing to update')

if ENABLE_UPDATE and PCL_exists and mismatchFound:
    from arcpy.da import UpdateCursor
    import time
    
    print('Updating ...')
    updateStartBaselineSeconds = int(time.time())
    updateCount = 0
    # for correctionDict in PIDN_CorrectionList:
    while PIDN_CorrectionList:
        correctionDict = PIDN_CorrectionList.pop(0)
        GID_ToMatch = correctionDict['SSAP_GlobalID']
        with UpdateCursor(ssapLayer, ['PIDN'],
          where_clause="GlobalID = '{}'".format(GID_ToMatch)) as updatePIDN:
            try:
                thisSSAP_PIDN = next(updatePIDN)
            except:
                print(f'''
                
                Unexpected error retrieving SSAP w/ GlobalID {GID_ToMatch}
                Stopping.

                You will need to re-enable updates to re-run this cell.

                ''')
                print('Stopping.')
                ENABLE_UPDATE = False
                raise
            newPIDN = correctionDict['CorrectPIDN']
            updatePIDN.updateRow( [newPIDN] )
        del updatePIDN # to be sure (with clause should handle this)
        addrString = addressStringFromCorrectionEntry(correctionDict)
        updateCount += 1
        print('Update {}: {}'.format(updateCount, addrString))
        if UPDATE_LIMIT and (updateCount >= UPDATE_LIMIT):
            print('Update limit reached.  Stopping.')
            break
        if UPDATE_MINUTES_MAXIMUM:
            updateMinutes = (time.time() - updateStartBaselineSeconds) / 60
            if updateMinutes > UPDATE_MINUTES_MAXIMUM:
                print(f'''
            Reached time limit for updates.
            You can re-enable via cells below and run this cell again
            to continue where you left off, for another {UPDATE_MINUTES_MAXIMUM} minutes.''')
                break
    print(f'{updateCount} SSAPs updated.')
    print('Done.')

ENABLE_UPDATE = False
print('You will need to re-enable updates to re-run this cell.')           


Updating ...
Update 1: 741 East Philadelphia Street YORK CITY 
Update 2: 58 North State Street YORK CITY 
Update 3: 672 East Philadelphia Street YORK CITY 
Update 4: 506 East Philadelphia Street YORK CITY 
Update 5: 242 East Prospect Street YORK CITY 
Update 6: 729 East Clarke Avenue YORK CITY 
Update 7: 244 Liberty Court YORK CITY 
Update 8: 709 West Princess Street YORK CITY 
Update 9: 658 East Market Street YORK CITY 
Update 10: 726 Glen Place YORK CITY 
Update limit reached.  Stopping.
10 SSAPs updated.
Done.
You will need to re-enable updates to re-run this cell.


## Enable *one* update as test (run following cell)

In [79]:
ENABLE_UPDATE = True
UPDATE_ONE_ONLY = True
print('Now run the cell above')

Now run the cell above


## Enable *ten* updates to time/benchmark

In [81]:
ENABLE_UPDATE = True
UPDATE_ONE_ONLY = False
UPDATE_TEN_ONLY = True

## Enable updates for 'x' minutes (set in below cell)
A time limit for updates behaves like the other (count) limits, in that if you re-run the update cell, it will continue updating where you left off.

In [44]:
UPDATE_MINUTES_MAXIMUM = 1
ENABLE_UPDATE = True
UPDATE_ONE_ONLY = False
UPDATE_TEN_ONLY = False

## Enable full updates (run following cell)

In [ ]:
ENABLE_UPDATE = True
UPDATE_ONE_ONLY = False
UPDATE_TEN_ONLY = False
UPDATE_MINUTES_MAXIMUM = None

## Disable updates

In [ ]:
ENABLE_UPDATE = False

# End of notebook